In [41]:
import api
import pandas as pd
import numpy as np
import datetime
from forex_python.converter import CurrencyRates
from forex_python.bitcoin import BtcConverter

In [34]:
def preprocessing(df, datum):
    dan = int(datum[:2])
    mesec = int(datum[4:5])
    leto = int(datum[6:])
    c = CurrencyRates()
    date_obj = datetime.datetime(leto, mesec, dan)
    eur_rate = c.get_rates('USD', date_obj)['EUR']
    b = BtcConverter()
    btc_rate = b.get_previous_price('USD', date_obj)
    pd.options.mode.chained_assignment = None  # default='warn'
    df = df.drop(df.columns[[0, 1]], axis=1)
    # priredim še enotni stolpec cen, kjer bodo vse cene v dolarjih 
    df['ceneDolarji'] = 0
    df['ceneDolarji'][df['druga'] == 'USD'] = df['price']
    df['ceneDolarji'][df['druga'] == 'EUR'] = df['price'] / eur_rate 
    df['ceneDolarji'][df['druga'] == 'BTC'] = df['price'] * btc_rate
    df['Leva'] = np.where(df['type']==0, df['druga'], df['prva'])
    df['Desna'] = np.where(df['type']==0, df['prva'], df['druga'])
    df['combo'] = df['Leva'] + df['Desna']
    df = df.drop(df.columns[[2, 3, 4]], axis=1)
    df['volume'] = df['ceneDolarji'] * df['amount']
    return df

In [26]:
def last(df):
    df_combo = df.groupby('combo')
    tmp = pd.DataFrame(df_combo.sum()['volume'])
    tmp.index.name = None
    tmp['combo'] = tmp.index
    tmp2 = pd.DataFrame(df_combo.sum()['amount'])
    tmp2.index.name = None
    tmp2['combo'] = tmp2.index
    tmp3 = pd.DataFrame(df_combo.mean()['ceneDolarji'])
    tmp3.index.name = None
    tmp3['combo'] = tmp3.index
    tmp4 = pd.DataFrame(df_combo.count()['price'])
    tmp4.index.name = None
    tmp4['combo'] = tmp4.index
    final = pd.merge(tmp, tmp2, on='combo')
    final = pd.merge(final, tmp3, on='combo')
    final = pd.merge(final, tmp4, on='combo')
    final.columns = ['volume', 'combo', 'nr_tokens', 'priceUSD', 'nr_txs']
    final['prva'] = final['combo'].str.extract('(^...)', expand=True)
    final['druga'] = final['combo'].str.extract('(...$)', expand=True)
    return final

In [27]:
def pametno_preberi(datum):
    pari = ['btcusd', 'btceur', 'eurusd', 'xrpusd', 'xrpeur', 'xrpbtc', 'ltcusd', 'ltceur', 'ltcbtc', 'ethusd', 'etheur', 'ethbtc']
    df = pd.DataFrame()
    for par in pari:
        ime = datum + '/' + par + '.csv'
        data = pd.read_csv(ime)
        data['prva'] = par[:3].upper()
        data['druga'] = par[3:].upper()
        df = df.append(data)
    return df

In [35]:
def tidy(datum):
    df = pametno_preberi(datum)
    df = preprocessing(df, datum)
    final = last(df)
    return final

In [39]:
for i in range(13,21):
    datum = str(i) + '.09.2017'
    df = tidy(datum)
    ime = datum + '/tidy.csv'
    df.to_csv(ime, index=False)
    print(datum + ' tidy ima!')

13.09.2017 tidy ima!
14.09.2017 tidy ima!
15.09.2017 tidy ima!
16.09.2017 tidy ima!
17.09.2017 tidy ima!
18.09.2017 tidy ima!
19.09.2017 tidy ima!
20.09.2017 tidy ima!


In [73]:
#volume in USD
datum = str(13) + '.09.2017'
ime = datum + '/tidy.csv'
df = pd.read_csv(ime)
df = df[df.columns[:2]]
df.columns = [datum, 'combo']
tabela = df

for i in range(14,21):
    datum = str(i) + '.09.2017'
    ime = datum + '/tidy.csv'
    df = pd.read_csv(ime)
    df = df[df.columns[:2]]
    df.columns = [datum, 'combo']
    tabela = pd.merge(tabela, df, how='inner', on='combo')
tabela = tabela.set_index('combo')
tabela = tabela.transpose()
tabela.loc['mean'] = tabela.mean()
tabela

combo,BTCETH,BTCEUR,BTCLTC,BTCUSD,BTCXRP,ETHBTC,ETHEUR,ETHUSD,EURBTC,EURETH,...,LTCEUR,LTCUSD,USDBTC,USDETH,USDEUR,USDLTC,USDXRP,XRPBTC,XRPEUR,XRPUSD
13.09.2017,7.729249e+05,1.251663e+07,2.595493e+05,6.443528e+07,363283.854168,1.070001e+06,4.220522e+06,6.061942e+06,1.035945e+07,6.545613e+06,...,1.440018e+06,3.618494e+06,5.113977e+07,7.389151e+06,1.280530e+06,3.477037e+06,1.545868e+06,313249.254385,1.130391e+06,1.849709e+06
14.09.2017,1.804145e+06,1.267328e+07,6.658221e+05,7.862361e+07,474212.933230,1.971037e+06,5.635807e+06,9.156655e+06,1.069932e+07,5.001855e+06,...,1.881914e+06,9.026657e+06,6.745258e+07,8.177196e+06,1.019588e+06,4.159123e+06,2.282303e+06,261836.220798,1.553788e+06,3.239545e+06
15.09.2017,3.041930e+06,1.821509e+07,2.088409e+06,9.862121e+07,600383.411017,3.637078e+06,8.943860e+06,1.021511e+07,2.211596e+07,8.433276e+06,...,3.951618e+06,1.122819e+07,1.045368e+08,1.240410e+07,2.444851e+06,7.964276e+06,4.386634e+06,759192.966838,2.115255e+06,3.839495e+06
16.09.2017,1.609841e+06,4.361773e+06,6.727939e+05,4.208319e+07,240190.033670,1.840577e+06,2.050914e+06,3.464412e+06,3.584049e+06,1.720902e+06,...,6.141721e+05,2.180405e+06,4.303307e+07,4.595378e+06,7.367895e+05,2.182359e+06,7.763366e+05,227228.431631,3.856409e+05,1.047401e+06
17.09.2017,1.074755e+06,2.979833e+06,2.849157e+05,2.108655e+07,231004.816064,1.110546e+06,1.872154e+06,1.818951e+06,3.091943e+06,1.267376e+06,...,5.547955e+05,8.867416e+05,2.134129e+07,3.084229e+06,5.886591e+05,9.233086e+05,9.553897e+05,276758.672097,2.543318e+05,7.124949e+05
18.09.2017,1.785108e+06,6.587975e+06,2.086334e+05,2.974516e+07,277314.779983,2.482569e+06,3.266119e+06,5.204078e+06,6.147028e+06,6.638912e+06,...,8.314001e+05,2.419165e+06,3.824481e+07,8.472685e+06,1.389293e+06,2.226334e+06,1.236635e+06,414861.681071,5.239844e+05,1.548601e+06
19.09.2017,9.320314e+05,3.242022e+06,3.640779e+05,3.253136e+07,145260.128810,1.011310e+06,2.227102e+06,3.661200e+06,4.311583e+06,3.381028e+06,...,9.625915e+05,2.825860e+06,2.787750e+07,3.174283e+06,8.027697e+05,2.042301e+06,8.281130e+05,170925.889357,4.087852e+05,9.568580e+05
20.09.2017,5.167761e+05,4.631479e+06,1.244401e+05,2.504422e+07,128122.298372,6.096113e+05,1.303690e+06,2.211292e+06,4.897360e+06,3.437877e+06,...,3.493365e+05,1.875302e+06,2.080829e+07,2.678989e+06,4.982087e+05,1.063429e+06,5.607627e+05,140566.603002,3.488057e+05,8.195661e+05
mean,1.442189e+06,8.151011e+06,5.835801e+05,4.902132e+07,307471.531914,1.716591e+06,3.690021e+06,5.224205e+06,8.150836e+06,4.553355e+06,...,1.323231e+06,4.257602e+06,4.680426e+07,6.247001e+06,1.095086e+06,3.004771e+06,1.571505e+06,320577.464897,8.401228e+05,1.751709e+06


In [72]:
#nr_txs
datum = str(13) + '.09.2017'
ime = datum + '/tidy.csv'
df = pd.read_csv(ime)
df = df[['combo', 'nr_txs']]
df.columns = ['combo', datum]
tabela = df

for i in range(14,21):
    datum = str(i) + '.09.2017'
    ime = datum + '/tidy.csv'
    df = pd.read_csv(ime)
    df = df[['combo', 'nr_txs']]
    df.columns = ['combo', datum]
    tabela = pd.merge(tabela, df, how='inner', on='combo')
tabela = tabela.set_index('combo')
tabela = tabela.transpose()
tabela.loc['mean'] = tabela.mean()
tabela.loc['min'] = tabela.min()
tabela.loc['max'] = tabela.max()
tabela

combo,BTCETH,BTCEUR,BTCLTC,BTCUSD,BTCXRP,ETHBTC,ETHEUR,ETHUSD,EURBTC,EURETH,...,LTCEUR,LTCUSD,USDBTC,USDETH,USDEUR,USDLTC,USDXRP,XRPBTC,XRPEUR,XRPUSD
13.09.2017,1432.0,6855.0,418.0,15282.0,840.0,1680.000,5008.0,7772.000,9436.000,6799.00,...,2707.0,3773.000,15560.0,7694.00,2633.00,3583.0,2543.000,618.000,2412.000,2592.00
14.09.2017,3122.0,8506.0,1177.0,21554.0,1116.0,3199.000,6867.0,9604.000,10702.000,5885.00,...,3713.0,7990.000,21982.0,8696.00,2384.00,4275.0,3315.000,685.000,2901.000,4031.00
15.09.2017,4512.0,13944.0,2453.0,31402.0,1501.0,5293.000,11154.0,13339.000,19832.000,10476.00,...,7361.0,10312.000,37481.0,13996.00,6123.00,8040.0,6398.000,1585.000,4049.000,4995.00
16.09.2017,3012.0,5389.0,736.0,10823.0,619.0,3270.000,4756.0,6342.000,6983.000,4395.00,...,1542.0,2193.000,15785.0,6318.00,1858.00,2244.0,1470.000,637.000,867.000,1741.00
17.09.2017,2154.0,4381.0,614.0,6725.0,656.0,2129.000,4275.0,3990.000,5473.000,3964.00,...,1890.0,1841.000,10638.0,5480.00,1779.00,1969.0,1600.000,720.000,750.000,1033.00
18.09.2017,2603.0,5633.0,469.0,11115.0,657.0,3284.000,5465.0,7899.000,8432.000,8475.00,...,2003.0,3351.000,15666.0,11125.00,2441.00,4030.0,2169.000,825.000,1068.000,1958.00
19.09.2017,1435.0,2993.0,443.0,8842.0,464.0,1507.000,3720.0,5367.000,5570.000,4063.00,...,1630.0,3296.000,10977.0,5033.00,1241.00,2741.0,1776.000,432.000,901.000,1346.00
20.09.2017,886.0,2603.0,298.0,6753.0,311.0,943.000,2079.0,3356.000,3975.000,2881.00,...,634.0,1987.000,8107.0,4060.00,767.00,1218.0,962.000,321.000,523.000,1182.00
mean,2394.5,6288.0,826.0,14062.0,770.5,2663.125,5415.5,7208.625,8800.375,5867.25,...,2685.0,4342.875,17024.5,7800.25,2403.25,3512.5,2529.125,727.875,1683.875,2359.75
min,886.0,2603.0,298.0,6725.0,311.0,943.000,2079.0,3356.000,3975.000,2881.00,...,634.0,1841.000,8107.0,4060.00,767.00,1218.0,962.000,321.000,523.000,1033.00
